## Import Library

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import os

In [2]:
import keras
import keras_vggface
from keras_vggface.vggface import VGGFace
import keras_vggface.utils
from keras_vggface.utils import preprocess_input
from keras import Model
from keras import layers
from keras.layers import Flatten, Input
from keras.layers.core import Dense
from keras.models import Sequential
from keras.optimizers import Adam

In [3]:
import mtcnn
from mtcnn.mtcnn import MTCNN
from scipy.spatial.distance import cosine
from keras.models import load_model

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.2),
])

In [ ]:
# Choose `avg_pool` as the last layer of your base model
last_layer = model.get_layer('avg_pool')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

## Load Your Model

In [4]:
model = load_model('./model/model-7-8105.h5')

In [ ]:
inputs = Input(shape=(224, 224, 3))

x = data_augmentation(inputs)
x = model(x)
x = Flatten(name='flatten')(x)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dense(1024, activation='relu', name='fc3')(x) 
x = Dense(1024, activation='relu', name='fc4')(x) 
x = Dense(512, activation='relu', name='fc5')(x)
x = Dense(512, activation='relu', name='fc6')(x) 
x = Dense(512, activation='relu', name='fc7')(x) 
x = Dense(256, activation='relu', name='fc8')(x)
x = Dense(256, activation='relu', name='fc9')(x)
out = Dense(105, name='classifier')(x)

In [ ]:
custom_model = Model(inputs, out)

## Make a Function for Face Detection 

In [5]:
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = plt.imread(filename)
    
    # create the detector, using default weights
    detector = MTCNN()
    
    # detect faces in the image
    results = detector.detect_faces(pixels)
    
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

## Verification Check

In [12]:
filenames = ["C:\\Users\\ACER\\Capstone\\verification\\shani1.jpg", 
             "C:\\Users\\ACER\\Capstone\\verification\\marsha1.jpg"] 

faces = [extract_face(f) for f in filenames]
samples = np.asarray(faces, "float32")
samples = preprocess_input(samples, version=2)

# perform prediction
embeddings = model.predict(samples)
thresh = 0.0015 
score = cosine(embeddings[0], embeddings[1])
if score <= thresh:
    print( " >face is a match (%.4f <= %.4f) " % (score, thresh))
else:
    print(" >face is NOT a match (%.4f > %.4f)" % (score, thresh))

1/1 [==============================] - 0s 107ms/step
 >face is NOT a match (0.0025 > 0.0015)
